# Frequency content of spike

A constant current input to the soma of the pyramidal cell model from Hay et al. (2011) leads to spiking with low firing rate. A time window containing a single spike is extracted, and the frequency content analyzed.


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import neuron
import LFPy
import elephant
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals.neural_simulations import return_hay_cell
import brainsignals.neural_simulations as ns

np.random.seed(12345)

In [ ]:
def insert_current_stimuli(cell):
    stim_params = {'amp': -0.4,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}
    ns.load_mechs_from_folder(ns.cell_models_folder)
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

In [ ]:
tstop = 150
dt = 2**-6

# Time window to extract spike from:
t0 = 100
t1 = 150

filt_dict_lf = {'highpass_freq': None,
                 'lowpass_freq': 300,
                 'order': 4,
                 'filter_function': 'filtfilt',
                 'fs': 1 / dt * 1000,
                 'axis': -1}

filt_dict_hf = {'highpass_freq': 300,
                 'lowpass_freq': None,
                 'order': 4,
                 'filter_function': 'filtfilt',
                 'fs': 1 / dt * 1000,
                 'axis': -1}

cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
syn, cell = insert_current_stimuli(cell)
cell.simulate(rec_imem=True, rec_vmem=True)

elec_params = dict(
            sigma = 0.3,      # extracellular conductivity
            x = np.array([20.]),
            y = np.array([0.0]),
            z = np.array([0.0]),
            method = 'root_as_point',
        )

elec = LFPy.RecExtElectrode(cell, **elec_params)
M_elec = elec.get_transformation_matrix()

t0_idx = np.argmin(np.abs(cell.tvec - t0))
t1_idx = np.argmin(np.abs(cell.tvec - t1))
eap = M_elec @ cell.imem[:, t0_idx:t1_idx] * 1000
vmem = cell.vmem[0, t0_idx:t1_idx]
tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]

# This is a trick to make signals start and end at zero. 
# It makes the FFTs look nicer
eap[0] -= np.linspace(eap[0, 0], eap[0, -1], len(eap[0]))
vmem_shift = vmem - np.linspace(vmem[0], vmem[-1], len(vmem))

eap_lf = elephant.signal_processing.butter(eap, **filt_dict_lf)
eap_hf = elephant.signal_processing.butter(eap, **filt_dict_hf)

freqs_vm, vm_psd = ns.return_freq_and_amplitude(tvec, vmem_shift)
freqs_eap, eap_psd = ns.return_freq_and_amplitude(tvec, eap)
freqs_eap, eap_psd_lf = ns.return_freq_and_amplitude(tvec, eap_lf)
freqs_eap, eap_psd_hf = ns.return_freq_and_amplitude(tvec, eap_hf)


In [ ]:
# The rest is plotting
plt.close("all")
fig = plt.figure(figsize=[6, 2.4])
fig.subplots_adjust(left=0.04, bottom=0.2, top=0.87, right=0.93,
                    hspace=0.5)

ax_vm = fig.add_subplot(131, title="membrane potential",
                        xlabel="", ylabel="", frameon=False, 
                        xticks=[], yticks=[],
                        xlim=[13, t1 - t0 - 20])

ax_eap = fig.add_subplot(132, title="extracellular spike",
                        xlabel="", ylabel="", frameon=False, 
                        xticks=[], yticks=[],
                        xlim=[13, t1 - t0 - 20])

ax_psd = fig.add_subplot(133, title="Fourier amplitudes",
                             xlim=[0, 2000], ylim=[-0.1, 6],
                             xlabel="frequency (Hz)", ylabel="µV / Hz")

ax_psd_vm = ax_psd.twinx()
ax_psd_vm.set_ylabel("mV / Hz")
ax_psd_vm.set_ylim([-0.1, 3.])

ax_vm.plot(tvec, vmem_shift[:], c='gray', lw=1.5)
l, = ax_eap.plot(tvec, eap[0], c='k', lw=1.5)
l_hf, = ax_eap.plot(tvec, eap_hf[0], c='b', lw=1.5)
l_lf, = ax_eap.plot(tvec, eap_lf[0], c='r', lw=1.5)

lines = [l, l_lf, l_hf]
line_names = ["original", "low-pass filtered (< 300 Hz)",
              "high-pass filtered  (> 300 Hz)"]

ax_vm.plot([25, 25], [20, 40], lw=2, c='k')
ax_vm.text(25.5, 30, "20 mV")

ax_vm.plot([25, 30], [20, 20], lw=2, c='k')
ax_vm.text(27, 17, "5 ms", ha="center", va="top")

ax_eap.plot([25, 25], [-50, -100], lw=2, c='k')
ax_eap.text(25.5, -75, "50 µV")

ax_eap.plot([25, 30], [-50, -50], lw=2, c='k')
ax_eap.text(27, -47, "5 ms", ha="center", va="bottom")

ax_psd_vm.plot(freqs_vm[1:], vm_psd[0][1:], c='gray', lw=1.5)
ax_psd.plot(freqs_eap[1:], eap_psd[0][1:], c='k', lw=1.5)
ax_psd.plot(freqs_eap[1:], eap_psd_hf[0][1:], c='b', lw=1.5)
ax_psd.plot(freqs_eap[1:], eap_psd_lf[0][1:], c='r', lw=1.5)

fig.legend(lines, line_names, loc="lower center", frameon=False, ncol=1)
mark_subplots([ax_vm, ax_eap, ax_psd])
simplify_axes([ax_vm, ax_eap])

fig.savefig("fig_spike_freq_content_amp.pdf")


In [ ]:
max_f_idx_eap = np.argmax(eap_psd[0])
max_eap_f = eap_psd[0][max_f_idx_eap]
print("Max EAP amplitude at: %d Hz" % freqs_eap[max_f_idx_eap])